To begin with we need to imports modules from Cirq and Mitiq libraries to facilitate the simulation of quantum circuits and apply error mitigation strategies such as Zero-Noise Extrapolation (ZNE), Probabilistic Error Cancellation (PEC), and Dynamical Decoupling (DDD).

Additionally, it imports specific functions for handling noise models, converting circuits to Mitiq format, and representing operations with local depolarizing noise. These functionalities are crucial for optimizing the performance and accuracy of quantum computations in the presence of noise.

In [11]:
import cirq

from cirq import DensityMatrixSimulator, depolarize # selecting simulator and noise
from mitiq.interface import convert_to_mitiq
from mitiq.pec.representations.depolarizing import represent_operations_in_circuit_with_local_depolarizing_noise

from mitiq import pec, zne, ddd
from mitiq.zne.scaling import fold_gates_at_random



This function, execute, simulates the execution of a quantum circuit with depolarizing noise and returns the expectation value of the density matrix corresponding to the |0⟩⟨0| state.

Args:
    circuit: The input quantum circuit to be executed.
    noise_level: The level of depolarizing noise to be applied to the circuit, defaults to 0.2.

Returns:
    The real part of the matrix element (0,0) of the final density matrix, representing the expectation value of the |0⟩⟨0| state prepared by the circuit executed with depolarizing noise.



In [12]:
def execute(circuit, noise_level=0.2):
    """Returns Tr[ρ |0⟩⟨0|] (expectation value)where ρ is the state prepared by the circuit
    executed with depolarizing noise.
    """
    # Replace with code based on your frontend and backend.
    mitiq_circuit, _ = convert_to_mitiq(circuit)
    
    # We add a simple noise model to simulate a noisy backend, depolarizing noise
    noisy_circuit = mitiq_circuit.with_noise(depolarize(p=noise_level))
    
    rho = DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix
    #DensityMatrixSimulator(*,dtype,noise):simulator for density matrices and noisy quantum circuits
    return rho[0, 0].real #real value of matrix element (0,0)

This code snippet demonstrates the simulation of a quantum circuit 'circuitA' with two X gates applied to qubit 'a0', representing the sequence X X. 

The noise level is set to 0.2, and the RichardsonFactory with scale factors [1.0, 2.0] is used for Zero-Noise Extrapolation (ZNE) extrapolation.

After constructing the circuit and executing it with and without noise, the noisy and ideal expectation values are calculated using the 'execute' function.

Additionally, the 'rule' variable is assigned the ddd.rules.xx rule, and 'repsA' represents the operations in 'circuitA' with local depolarizing noise at the specified noise level.



In [13]:
a0 = cirq.q("a")
# 5,1,3
# 1 -> 5
# X X
ops4 = [cirq.X(a0), cirq.X(a0)]
noise_level = 0.2
factory = zne.RichardsonFactory([1.0, 2.0])
circuitA = cirq.Circuit(ops4)
noisy_valueA= execute(circuitA, noise_level)
ideal_valueA= execute(circuitA, noise_level=0.0)
rule = ddd.rules.xx
repsA = represent_operations_in_circuit_with_local_depolarizing_noise(circuitA, noise_level)

This code snippet demonstrates the use of Probabilistic Error Cancellation (PEC) to mitigate errors in quantum circuit execution.

The 'pec_executor' is created using 'pec.mitigate_executor' function, which takes the 'execute' function as input along with representations of operations in 'circuitA' with local depolarizing noise ('repsA').

The 'pec_executor' is then used to execute 'circuitA' with PEC, resulting in 'pec_valueA'.

Finally, the error reduction achieved by PEC is evaluated by comparing the noisy and ideal expectation values obtained with and without PEC, respectively. The absolute differences are printed to the console.


In [14]:
pec_executor = pec.mitigate_executor(execute, representations=repsA)
pec_valueA = pec_executor(circuitA)
print(f"Error without PEC:Circuit3: {abs(ideal_valueA - noisy_valueA) :.5f}")
print(f"Error with PEC:Circuit3:    {abs(ideal_valueA - pec_valueA) :.5f}")


Error without PEC:Circuit3: 0.23111
Error with PEC:Circuit3:    0.02888



This code snippet demonstrates the application of Zero-Noise Extrapolation (ZNE) to mitigate errors in quantum circuit execution.

The 'zne_executor' is created using 'zne.mitigate_executor' function, which takes the 'execute' function as input along with the ZNE factory ('factory') specifying scale factors for extrapolation.

The 'zne_executor' is then used to execute 'circuitA' with ZNE, resulting in 'zne_valueA'.

Finally, the error reduction achieved by ZNE is evaluated by comparing the noisy and ideal expectation values obtained with and without ZNE, respectively. The absolute differences are printed to the console.


In [15]:
zne_executor = zne.mitigate_executor(execute, factory=factory)
zne_valueA = zne_executor(circuitA)

print(f"Error without ZNE:Circuit3: {abs(ideal_valueA - noisy_valueA) :.5f}")
print(f"Error with ZNE:Circuit3:    {abs(ideal_valueA - zne_valueA) :.5f}")

Error without ZNE:Circuit3: 0.23111
Error with ZNE:Circuit3:    0.10682


/opt/homebrew/anaconda3/envs/qrize/lib/python3.11/site-packages/mitiq/zne/inference.py:96: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(



This code snippet demonstrates the application of Dynamical Decoupling (DDD) to mitigate errors in quantum circuit execution.

The 'ddd_executor' is created using the 'ddd.mitigate_executor' function, which takes the 'execute' function as input along with the specified DDD rule ('rule').

The 'ddd_executor' is then used to execute 'circuitA' with DDD, resulting in 'ddd_value'.

Finally, the error reduction achieved by DDD is evaluated by comparing the noisy and ideal expectation values obtained with and without DDD, respectively. The absolute differences are printed to the console.


In [16]:
ddd_executor = ddd.mitigate_executor(executor=execute,rule = rule)
ddd_value = ddd_executor(circuitA)

print(f"Error without DDD:Circuit3: {abs(ideal_valueA - noisy_valueA) :.5f}")
print(f"Error with DDD:Circuit3:    {abs(ideal_valueA - ddd_value) :.5f}")

Error without DDD:Circuit3: 0.23111
Error with DDD:Circuit3:    0.23111



This code snippet demonstrates the combined application of Probabilistic Error Cancellation (PEC) and Dynamical Decoupling (DDD) to mitigate errors in quantum circuit execution.

The 'pec_ddd_combined_executor' is created using the 'ddd.mitigate_executor' function, which takes the 'pec_executor' as input along with the specified DDD rule ('rule').

The 'pec_ddd_combined_executor' is then used to execute the PEC mitigated circuit with DDD, resulting in improved error mitigation.


In [17]:
pec_ddd_combined_executor = ddd.mitigate_executor(executor=pec_executor,rule = rule)


This code snippet demonstrates the combined application of Probabilistic Error Cancellation (PEC) and Zero-Noise Extrapolation (ZNE) to mitigate errors in quantum circuit execution.

The 'zne_executor' is created using the 'zne.mitigate_executor' function, which takes the 'pec_executor' as input along with the ZNE factory ('factory') specifying scale factors for extrapolation.

The 'zne_executor' is then used to execute 'circuitA' with the combined PEC and ZNE techniques, resulting in 'combined_value'.

Finally, the error reduction achieved by the combined approach is evaluated by comparing the noisy and ideal expectation values obtained with and without the combined approach, respectively. The absolute differences are printed to the console.


In [18]:
zne_executor = zne.mitigate_executor(pec_executor, factory=factory)
combined_value = zne_executor(circuitA)

print(f"Error without COMBINED:Circuit3: {abs(ideal_valueA - noisy_valueA) :.5f}")
print(f"Error with COMBINED:Circuit3:    {abs(ideal_valueA - combined_value) :.5f}")

/opt/homebrew/anaconda3/envs/qrize/lib/python3.11/site-packages/mitiq/zne/inference.py:96: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(


Error without COMBINED:Circuit3: 0.23111
Error with COMBINED:Circuit3:    0.02374



This code snippet demonstrates the combined application of Zero-Noise Extrapolation (ZNE) with a previously combined PEC and DDD technique to further mitigate errors in quantum circuit execution.

The 'zne_executor' is created using the 'zne.mitigate_executor' function, which takes the previously combined 'pec_ddd_combined_executor' as input along with the ZNE factory ('factory') specifying scale factors for extrapolation.

The 'zne_executor' is then used to execute 'circuitA' with the combined PEC, DDD, and ZNE techniques, resulting in 'combined_value'.

Finally, the error reduction achieved by the combined approach is evaluated by comparing the noisy and ideal expectation values obtained with and without the combined approach, respectively. The absolute differences are printed to the console.


In [19]:
zne_executor = zne.mitigate_executor(pec_ddd_combined_executor, factory=factory)
combined_value = zne_executor(circuitA)

print(f"Error without COMBINED:Circuit3: {abs(ideal_valueA - noisy_valueA) :.5f}")
print(f"Error with COMBINED:Circuit3:    {abs(ideal_valueA - combined_value) :.5f}")

/opt/homebrew/anaconda3/envs/qrize/lib/python3.11/site-packages/mitiq/zne/inference.py:96: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(


Error without COMBINED:Circuit3: 0.23111
Error with COMBINED:Circuit3:    0.01252


This function launches the specified error mitigation technique (executor) with additional arguments (executor_args) applied to the given quantum circuit.

Parameters:
- executor: The error mitigation technique to be launched.
- types: The type of error mitigation technique being applied (e.g., PEC, DDD, ZNE, or a combination).
- circuit: The quantum circuit to be executed.
- **executor_args: Additional arguments for configuring the error mitigation technique.

Returns:
- None

This function creates a ZNE executor by combining the specified executor with additional arguments, and then applies it to the given circuit to obtain a mitigated value ('combined_value'). The error reduction achieved by the specified error mitigation technique is evaluated by comparing the noisy and ideal expectation values obtained with and without the technique, respectively. The absolute differences are printed to the console.


In [1]:
def launch(executor, types, circuit, **executor_args):
   zne_executor = zne.mitigate_executor(executor=executor, **executor_args)
   combined_value = zne_executor(circuit)

   print(f"Error without {types}:Circuit3: {abs(ideal_valueA - noisy_valueA) :.5f}")
   print(f"Error with {types}:Circuit3:    {abs(ideal_valueA - combined_value) :.5f}")